In [2]:
def get_sales_related_features(target_variable):
    
    """
    Generating sales related features 
    
    Args:
        target_variable: Pandas DataFrame
            Target Variable
    
    Returns:
        sales_related_features: Pandas DataFrame
            Sales related features
    """
    
    sales_related_features = target_variable.copy()
    sales_related_features = sales_related_features.sort_values(by="week").reset_index(drop=True)
    
    window = sales_related_features.groupby(["outlet_code", "item_department"]).rolling(window=4)
    
    sales_related_features["fe_avg_4_week_sales"] = window["total_sales_qty"].mean().reset_index(
        level=[0, 1,2],drop=True)
    sales_related_features["fe_4_weeks_std_dev_weekly"] = window["total_sales_qty"].std().reset_index(
        level=[0, 1,2], drop=True)
    sales_related_features["fe_4_weeks_weekly_min_sales"]=window["total_sales_qty"].min().reset_index(
        level=[0, 1,2], drop=True)
    sales_related_features["fe_4_weeks_weekly_max_sales"]=window["total_sales_qty"].max().reset_index(
        level=[0, 1,2], drop=True)
    
    sales_related_features = sales_related_features.sort_values(by=["outlet_code", "item_department",
                                                                    "week"])
    # Previous week sales
    sales_related_features['previous_week_sales']=sales_related_features.groupby(['outlet_code',
                                                                                  'item_department']
                                                                                )['total_sales_qty'].shift(1)
    # Previous 2 weeks sales
    sales_related_features['prev_2_weeks_sales']=sales_related_features.groupby(['outlet_code', 'item_department'])['total_sales_qty'].rolling(window=2).sum().reset_index(level=[0, 1], drop=True)
    
    # Previous 3 weeks sales
    sales_related_features['prev_3_weeks_sales'] = sales_related_features.groupby(['outlet_code', 'item_department'])['total_sales_qty'].rolling(window=3).sum().reset_index(level=[0, 1], drop=True)
    
    # Previous month sales
    sales_related_features['prev_month_sales'] = sales_related_features.groupby(['outlet_code', 'item_department'])['total_sales_qty'].rolling(window=4).sum().reset_index(level=[0, 1], drop=True)
    
    # Outlet-wise minimum and maximum sales
    sales_related_features['outlet_min_sales'] = sales_related_features.groupby('outlet_code')['total_sales_qty'].transform('min')
    sales_related_features['outlet_max_sales'] = sales_related_features.groupby('outlet_code')['total_sales_qty'].transform('max')

    # Calculate the weekly sales change
    sales_related_features['fe_sales_change_vs_next_week'] = sales_related_features['sales_next_week'] - sales_related_features['total_sales_qty']
    
    # Calculate the difference between current week's sales and previous week's sales
    sales_related_features['fe_sales_change_vs_previous_week'] = sales_related_features['total_sales_qty'].diff()
    #sales_related_features['fe_sales_change_vs_previous_week'].fillna(0, inplace=True)
    
    # Calculate the ratio of sales to the maximum sales in the dataset
    sales_related_features['fe_sales_to_max_sales_ratio'] = sales_related_features['total_sales_qty'] / sales_related_features['total_sales_qty'].max()
    
    # Calculate the cumulative sales up to the current week
    sales_related_features['fe_cumulative_sales'] = sales_related_features['total_sales_qty'].cumsum()
    
    sales_related_features = sales_related_features[[
    'outlet_code',
    'item_department',
    'week',
    'fe_avg_4_week_sales',
    'fe_4_weeks_std_dev_weekly',
    'fe_4_weeks_weekly_min_sales',
    'fe_4_weeks_weekly_max_sales',
    'previous_week_sales',
    'prev_2_weeks_sales',
    'prev_3_weeks_sales',
    'prev_month_sales',
    'outlet_min_sales',
    'outlet_max_sales',
    'fe_sales_change_vs_next_week',
    'fe_sales_change_vs_previous_week',
    'fe_sales_to_max_sales_ratio',
    'fe_cumulative_sales'
    ]]
    
    return sales_related_features
